In [2]:
import requests
import pandas as pd
import pprint
from datetime import date
import json
import glob

In [3]:
#dataGame = pd.json_normalize(d,record_path= ['rosterSpots'])
#dataGame.head(3)[['teamId','firstName.default']]
#dataGame[dataGame['teamId']==12][['firstName.default','lastName.default']]

In [3]:
df = pd.DataFrame()
for file in glob.glob('games/2023*.json'):
    #print(file)
    with open(f'{file}') as json_file:
        data = json.load(json_file)
    dataGame = pd.json_normalize(data,record_path= ['plays'])
    dataGame['game']=file
    df=pd.concat([df,dataGame])
# df.describe()
df.head(5)[['eventId','periodDescriptor.number','timeInPeriod','typeDescKey','details.winningPlayerId','details.xCoord','details.yCoord','details.shootingPlayerId','details.eventOwnerTeamId']]
# df.shape
players=pd.read_csv('games/players.csv')
players

,playerId,teamId,sweaterNumber,positionCode,firstName.default,lastName.default
0,8470600,25,20,D,Ryan,Suter
1,8470604,5,77,C,Jeff,Carter
2,8470610,21,9,L,Zach,Parise
3,8470613,12,8,D,Brent,Burns
4,8470794,25,16,C,Joe,Pavelski
...,...,...,...,...,...,...
657,8484144,16,98,C,Connor,Bedard
658,8484145,7,9,L,Zach,Benson
659,8484293,12,60,G,Yaniv,Perets
660,8484304,13,26,D,Uvis,Balinskis


In [1]:
df[(df['details.shootingPlayerId']==8480830)]
df.notna

<bound method DataFrame.notna of      eventId timeInPeriod  ... period details.servedByPlayerId
0       52.0        00:00  ...    NaN                      NaN
1       51.0        00:00  ...    NaN                      NaN
2      151.0        00:17  ...    NaN                      NaN
3      105.0        00:41  ...    NaN                      NaN
4      154.0        01:06  ...    NaN                      NaN
..       ...          ...  ...    ...                      ...
275    529.0        19:30  ...    3.0                      NaN
276    802.0        19:30  ...    3.0                      NaN
277   1023.0        19:55  ...    3.0                      NaN
278     50.0        20:00  ...    3.0                      NaN
279    604.0        20:00  ...    3.0                      NaN

[15462 rows x 43 columns]>

## types of records

In [9]:
df["typeDescKey"].unique()
#dataGame.dtypes

array(['period-start', 'faceoff', 'hit', 'blocked-shot', 'stoppage',
       'shot-on-goal', 'delayed-penalty', 'penalty', 'giveaway',
       'takeaway', 'missed-shot', 'goal', 'period-end', 'game-end',
       'shootout-complete'], dtype=object)

## Goals

In [4]:
goals=df[(df["typeDescKey"]=="missed-shot") | (df["typeDescKey"]=="blocked-shot")]
# grouped=goals.groupby('team.name')
goals.head(3)[['timeInPeriod','details.reason']]
#goals[['team.name','coordinates.x','coordinates.y']]
goals

,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,periodDescriptor.number,periodDescriptor.periodType,details.eventOwnerTeamId,details.losingPlayerId,details.winningPlayerId,details.xCoord,details.yCoord,details.zoneCode,details.hittingPlayerId,details.hitteePlayerId,details.blockingPlayerId,details.shootingPlayerId,details.reason,details.shotType,details.goalieInNetId,details.awaySOG,details.homeSOG,details.typeCode,details.descKey,details.duration,details.committedByPlayerId,details.drawnByPlayerId,details.playerId,details.scoringPlayerId,details.scoringPlayerTotal,details.assist1PlayerId,details.assist1PlayerTotal,details.awayScore,details.homeScore,details.secondaryReason,details.assist2PlayerId,details.assist2PlayerTotal,game,period,details.servedByPlayerId
3,105.0,00:41,19:19,1551,right,508.0,blocked-shot,15.0,1.0,REG,12.0,NaN,NaN,34.0,-4.0,D,NaN,NaN,8476921.0,8478430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023021024.json,NaN,NaN
6,8.0,01:16,18:44,1551,right,508.0,blocked-shot,25.0,1.0,REG,20.0,NaN,NaN,-52.0,36.0,D,NaN,NaN,8478430.0,8480039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023021024.json,NaN,NaN
12,122.0,01:45,18:15,1451,right,508.0,blocked-shot,37.0,1.0,REG,20.0,NaN,NaN,-49.0,7.0,D,NaN,NaN,8479291.0,8470613.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023021024.json,NaN,NaN
28,218.0,05:46,14:14,1551,right,507.0,missed-shot,92.0,1.0,REG,12.0,NaN,NaN,-27.0,30.0,O,NaN,NaN,NaN,8477488.0,wide-left,snap,8478435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023021024.json,NaN,NaN
35,228.0,06:58,13:02,1551,right,508.0,blocked-shot,104.0,1.0,REG,20.0,NaN,NaN,-69.0,-35.0,D,NaN,NaN,8476399.0,8476958.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023021024.json,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,932.0,11:11,08:49,1551,right,507.0,missed-shot,652.0,3.0,REG,12.0,NaN,NaN,-76.0,-19.0,O,NaN,NaN,NaN,8478427.0,wide-left,wrist,8476883.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023020303.json,3.0,NaN
245,935.0,11:20,08:40,1551,right,507.0,missed-shot,656.0,3.0,REG,14.0,NaN,NaN,54.0,19.0,O,NaN,NaN,NaN,8476453.0,wide-left,wrist,8477293.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023020303.json,3.0,NaN
254,49.0,14:02,05:58,1451,right,507.0,missed-shot,687.0,3.0,REG,12.0,NaN,NaN,-79.0,2.0,O,NaN,NaN,NaN,8482093.0,high-and-wide-left,tip-in,8476883.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023020303.json,3.0,NaN
258,974.0,15:25,04:35,1551,right,508.0,blocked-shot,700.0,3.0,REG,12.0,NaN,NaN,85.0,-5.0,D,NaN,NaN,8476869.0,8479661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,games/2023020303.json,3.0,NaN


In [5]:
goals=df[(df["typeDescKey"]=="goal") & (df["details.eventOwnerTeamId"]==12)][['timeInPeriod','periodDescriptor.number','details.xCoord','details.yCoord','details.shotType','details.scoringPlayerId','details.assist1PlayerId','details.eventOwnerTeamId','game']]
goals

,timeInPeriod,periodDescriptor.number,details.xCoord,details.yCoord,details.shotType,details.scoringPlayerId,details.assist1PlayerId,details.eventOwnerTeamId,game
32,06:30,1.0,-79.0,4.0,wrist,8476921.0,8478427.0,12.0,games/2023021024.json
113,17:43,1.0,-68.0,-9.0,snap,8478427.0,8476882.0,12.0,games/2023021024.json
130,00:17,2.0,39.0,-27.0,slap,8478970.0,8482093.0,12.0,games/2023021024.json
136,01:15,2.0,61.0,-15.0,slap,8480830.0,8478427.0,12.0,games/2023021024.json
223,13:55,2.0,39.0,29.0,slap,8470613.0,8476958.0,12.0,games/2023021024.json
...,...,...,...,...,...,...,...,...,...
53,12:43,1.0,-43.0,29.0,wrist,8477950.0,NaN,12.0,games/2023020180.json
224,07:54,3.0,-73.0,15.0,snap,8476869.0,8478427.0,12.0,games/2023020180.json
287,01:30,4.0,85.0,-1.0,tip-in,8480039.0,8478427.0,12.0,games/2023020180.json
78,18:12,1.0,-83.0,9.0,snap,8476474.0,8480835.0,12.0,games/2023020303.json


In [6]:
scores=goals.merge(players, how='left', left_on='details.scoringPlayerId', right_on='playerId')
# scores.describe
#df.groupby('team')['points'].mean()
grouped = scores.groupby(['playerId','firstName.default'])[['details.xCoord','details.yCoord']].mean()
scores['count']=scores['playerId'].value_counts()
scores

,timeInPeriod,periodDescriptor.number,details.xCoord,details.yCoord,details.shotType,details.scoringPlayerId,details.assist1PlayerId,details.eventOwnerTeamId,game,playerId,teamId,sweaterNumber,positionCode,firstName.default,lastName.default,count
0,06:30,1.0,-79.0,4.0,wrist,8476921.0,8478427.0,12.0,games/2023021024.json,8476921,12,48,L,Jordan,Martinook,NaN
1,17:43,1.0,-68.0,-9.0,snap,8478427.0,8476882.0,12.0,games/2023021024.json,8478427,12,20,C,Sebastian,Aho,NaN
2,00:17,2.0,39.0,-27.0,slap,8478970.0,8482093.0,12.0,games/2023021024.json,8478970,12,5,D,Jalen,Chatfield,NaN
3,01:15,2.0,61.0,-15.0,slap,8480830.0,8478427.0,12.0,games/2023021024.json,8480830,12,37,R,Andrei,Svechnikov,NaN
4,13:55,2.0,39.0,29.0,slap,8470613.0,8476958.0,12.0,games/2023021024.json,8470613,12,8,D,Brent,Burns,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,07:54,3.0,-73.0,15.0,snap,8476869.0,8478427.0,12.0,games/2023020180.json,8476869,12,76,D,Brady,Skjei,NaN
181,01:30,4.0,85.0,-1.0,tip-in,8480039.0,8478427.0,12.0,games/2023020180.json,8480039,12,88,C,Martin,Necas,NaN
182,18:12,1.0,-83.0,9.0,snap,8476474.0,8480835.0,12.0,games/2023020303.json,8476474,12,23,R,Stefan,Noesen,NaN
183,14:45,2.0,52.0,8.0,slap,8478047.0,8480830.0,12.0,games/2023020303.json,8478047,12,58,L,Michael,Bunting,NaN


In [10]:
scores[(scores['playerId']==8480830)]

,timeInPeriod,periodDescriptor.number,details.xCoord,details.yCoord,details.shotType,details.scoringPlayerId,details.assist1PlayerId,details.eventOwnerTeamId,game,playerId,teamId,sweaterNumber,positionCode,firstName.default,lastName.default,count
3,01:15,2.0,61.0,-15.0,slap,8480830.0,8478427.0,12.0,games/2023021024.json,8480830,12,37,R,Andrei,Svechnikov,NaN
8,16:15,1.0,-76.0,17.0,snap,8480830.0,8478427.0,12.0,games/2023020639.json,8480830,12,37,R,Andrei,Svechnikov,NaN
20,12:41,3.0,72.0,-21.0,slap,8480830.0,8478427.0,12.0,games/2023020603.json,8480830,12,37,R,Andrei,Svechnikov,NaN
40,01:21,2.0,-68.0,7.0,backhand,8480830.0,8482093.0,12.0,games/2023020862.json,8480830,12,37,R,Andrei,Svechnikov,NaN
65,18:57,3.0,-60.0,-26.0,snap,8480830.0,8476882.0,12.0,games/2023020994.json,8480830,12,37,R,Andrei,Svechnikov,NaN
112,19:29,1.0,81.0,-13.0,wrist,8480830.0,8478427.0,12.0,games/2023020575.json,8480830,12,37,R,Andrei,Svechnikov,NaN
115,02:56,3.0,79.0,-6.0,wrist,8480830.0,8476869.0,12.0,games/2023020575.json,8480830,12,37,R,Andrei,Svechnikov,NaN
152,10:46,3.0,-82.0,4.0,wrist,8480830.0,8478047.0,12.0,games/2023020702.json,8480830,12,37,R,Andrei,Svechnikov,NaN
174,15:42,3.0,69.0,-38.0,wrist,8480830.0,8476958.0,12.0,games/2023021046.json,8480830,12,37,R,Andrei,Svechnikov,NaN
177,18:49,3.0,-73.0,2.0,wrist,8480830.0,8476882.0,12.0,games/2023021011.json,8480830,12,37,R,Andrei,Svechnikov,NaN
